In [ ]:
!pip uninstall -y torch -q
!pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm -q
# keep data in float16 to avoid OOM
file_path = '/opt/conda/lib/python3.10/site-packages/transformers/models/llama/modeling_llama.py'
with open(file_path, 'r') as file:
    file_contents = file.readlines()
file_contents = [line for line in file_contents if "logits = logits.float()" not in line]
with open(file_path, 'w') as file:
    file.writelines(file_contents)

In [ ]:
from vllm import LLM, SamplingParams
import numpy as np
from transformers import LlamaForSequenceClassification
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)
llm = LLM(model="/kaggle/input/deepseek-math",
          dtype='half',
          enforce_eager=True,
          gpu_memory_utilization=0.99,
          swap_space=4,
          max_model_len=2048,
          kv_cache_dtype="fp8_e5m2",
          tensor_parallel_size=1)

tokenizer = llm.get_tokenizer()

prm_tokenizer = tokenizer
prm_model = LlamaForSequenceClassification.from_pretrained('/kaggle/input/prm-code',\
                                                    num_labels=1,\
                                                    device_map="cuda:1",
                                                    torch_dtype="auto",
                                                    ignore_mismatched_sizes=True,
                                                    ).eval()

base_model = prm_model.model
prm_model.score.load_state_dict(torch.load('/kaggle/input/prm-code/model_score3_code.pth'))

In [ ]:
import aimo
env = aimo.make_env()
iter_test = env.iter_test()

In [ ]:
logit2prob = lambda x: 1/(1+np.exp(-x))
def eval_prm(candidates):
    all_log_probs = []
    for i in range(len(candidates)):
        input_ids = prm_tokenizer.encode(candidates[i], return_tensors="pt").to("cuda:1")
        with torch.no_grad():
            hidden_states = base_model(input_ids)[0][:,-1] # 1,l,d -> 1,d
            logits = prm_model.score(hidden_states)[0]
        all_log_probs.append(logit2prob(logits.item()))
    return all_log_probs

In [ ]:
stop_words = [tokenizer.eos_token,"```output","```Output","```output\n","```Output\n","```\nOutput" , ")\n```" , "``````output"]
# stop_words.append("\n")
sampling_params = SamplingParams(temperature=1,
                                 max_tokens=256,
                                #  min_tokens=32,
                                 stop=stop_words,
                                 include_stop_str_in_output=True
                                )

def gen_prompt_codeIn1(problem):
    return f"""
Problem:{problem}\n
Let's reason step by step and write Python (sympy) code to solve the problem, using brute force enumeration if necessary. 
Limit the search space using logical deductions. The code should be enclosed between ```python\n actual code...``` and should only print the final answer.
"""

n = 3 # beams
n_sol = 3
samples = 7
max_depth = 20
max_pct = 0.88
timeout = 7
len_limit = 49


all_prompts = []
total_paths = []
total_answers = []

def is_integer(num):
    if isinstance(num, float):
        return num.is_integer()
    elif isinstance(num, int):
        return True
    else:
        return False
    
def is_between_0_and_999(num):
    return 0 <= num <= 999

import re
def extract_number(text):
    patterns = [
        r'[Tt]he answer is.*\\boxed\{(.*?)\}',
        r"[Tt]he answer is[:\s]*\$([0-9]+)\$",
        r"[Tt]he answer is[:\s]*([0-9]+)"
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1)
    return 'parse err'

def repeat_elements(lst, k):
    return [i for i in lst for _ in range(k)]

def filter_input(batch_response,current_level_node):
    # one question filter
    prm_inputs = []
    for candidate,parent in zip(batch_response,current_level_node):
        if candidate.outputs[0].text not in parent:
            prm_input = parent + candidate.outputs[0].text
            prm_inputs.append(prm_input)
    # Get the indices of unique elements in prm_inputs
    unique_indices = [i for i, x in enumerate(prm_inputs) if prm_inputs.index(x) == i]
    prm_inputs = [prm_inputs[i] for i in unique_indices]
    return prm_inputs

def IsFinished(node):
    matches = re.findall(r'print\(([^)]*)\)', node)
    return len(matches)>0

def get_next_node(prm_inputs,prm_scores):
    # need to update completed_paths in-place
    if len(prm_inputs) == 0: return []
    next_level_nodes = []
    combined = list(zip(prm_inputs,prm_scores))
    combined.sort(key=lambda x: x[1], reverse=True)  # Sort nodes by their scores
    max_score = combined[0][1]
    for node,score in combined:
        finish = IsFinished(node)
        if finish: # finished
            if score > max_score * max_pct:
                completed_paths.append((score,node))
        else: # not inished
            if (len(next_level_nodes) < n) and (score > max_score * max_pct):
                next_level_nodes.append(node)
    return next_level_nodes

import subprocess
import sys
def agg_code(paths):
    paths = [p for p in paths if p]
    paths.sort(key=lambda x: x[0], reverse=True)
    for path in paths:# path (score,node)
        if path:# not None
            input = path[1]
            if input[-12:]=="print(result": # stop token was not included. print(result) might miss a ")"
                input += ")"
            splits = input.split('```')
            if len(splits) <= 3: # 3 bc prompt include two ```
                continue
            code = "from sympy import *\n" + input.split('```')[3][7:] 
            if len(code) < len_limit: continue # ignore very short answer

            # execute code
            with open('code.py', 'w') as fout:
                fout.write(code)
            # timeout err
            try:
                process = subprocess.run([sys.executable, 'code.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=timeout)
            except subprocess.TimeoutExpired:
                continue
            if process.stderr:# code.py err
                continue
            else:
                stdout = process.stdout.decode('utf8')
                try:
                    answer = eval(stdout)
                    if is_integer(answer) and is_between_0_and_999(answer):
                        return int(answer)
                    else:
                        continue
                except:
                    continue
    return 37

for test, sample_submission in iter_test:
    problem = test['problem'].values[0]

    messages = [
        {
            "role": "user",
            "content": gen_prompt_codeIn1(problem)
        }
    ]

    base_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )
    current_level = 1

    current_level_nodes = [base_prompt]
    completed_paths = []
    completed_path_splits = []
    try:
        while (len(completed_paths) < n_sol) and (current_level < max_depth) and (current_level_nodes):
            current_level_nodes = repeat_elements(current_level_nodes,samples)
            batch_responses = llm.generate(current_level_nodes, sampling_params)
            prm_inputs = filter_input(batch_responses,current_level_nodes)
            prm_scores = eval_prm(prm_inputs)
            current_level_nodes = get_next_node(prm_inputs,prm_scores)
        sample_submission['answer'] = agg_code(completed_paths)
    except:
        sample_submission['answer'] = 37
    env.predict(sample_submission)

In [ ]:
# total_paths
# len(set(current_level_nodes)),len(current_level_nodes),len(set(prm_inputs)),len(prm_inputs)